In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from BuildDatabase import BuildDatabase

pd.options.plotting.backend = 'plotly'

config = dict(
    low=0.5,
    high=8.0,
    sim1=0.7,
    sim2=0.9,
    snr_t=20,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0,
)

worker = BuildDatabase(
    output_dir='data-2022-09-23/',
    config=config,
    win_len=1024,
    fs=125,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)
windows, n_excluded, sim, snr, hr, abp_max, abp_min = worker.run()

with open('data.pkl', 'wb') as f:
    pkl.dump([windows, n_excluded, sim, snr, hr, abp_max, abp_min], f)

In [ ]:
from DataEvaluator import DataEvaluator

with open('data.pkl', 'rb') as f:
    windows, n_excluded, sim, snr, hr, abp_max, abp_min = pkl.load(f)

eval = DataEvaluator(
    sim=sim,
    snr=snr,
    hr=hr,
    n_excluded=n_excluded,
)

In [ ]:
fig = eval.evaluate_sim(bins=25)
fig.show()

In [ ]:
fig = eval.evaluate_hr(bins=50)
fig.show()